# Multi-resolution routing

In [ ]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import FixedEmbeddingComposite
import dwave_networkx as dnx
import networkx as nx
from itertools import product as prd
import matplotlib.pyplot as plt

%matplotlib inline

import sys
from pathlib import Path
pp = str(Path('.').absolute().parent)
if pp not in sys.path:
    sys.path.append(pp)

In [ ]:
from refactor.essentials import (
    RectGridGraph, create_qubo, SA, optimize_qannealer,
    is_this_an_answer,
)
from qpr.quantum_utils import find_embedding_minorminer

In [ ]:
#int(len(G.nodes())/(9))
#list(G.edges())
#G.size()

In [ ]:
class cluster_graph(nx.Graph):
    """Cluster is a graph whose nodes are graphs themselves"""
    def __init__(self, sub_graphs = None, cluster_edges={((0,0), (0,1)): [[(0,2), (0,1)], [(1,2), (0,1)], [(2,2), (0,1)]]}):
        super().__init__(nx.Graph())
        cluster_edges_keys = [list(item) for item in cluster_edges.keys()]
        self.add_edges_from(cluster_edges_keys) # Nodes of this graph are graphs
        self.inter_node_connections = cluster_edges
        self.sub_graph = sub_graphs
    @classmethod
    def from_graph(cls, graph):
        pass
    @classmethod
    def from_rect_graph(cls, graph_nrow, graph_ncol, cluster_nrow, cluster_ncol):
        assert (graph_nrow%cluster_nrow == 0),"Error"
        assert (graph_ncol%cluster_ncol == 0),"Error"
        edges = RectGridGraph(cluster_nrow, cluster_ncol).edges
        G = RectGridGraph(graph_nrow, graph_ncol)
        cluster_edges = {}
        row = int(graph_nrow / cluster_nrow)
        col = int(graph_ncol / cluster_ncol)
        for item in edges:
            cluster_edges[item] = []
            if item[0][0] == item[1][0] and item[0][0] < cluster_ncol-1:
                for i in range(0, col):
                    cluster_edges[item].append(((item[0][0] * col + i, item[1][1] * row - 1), (item[0][0] * col + i, item[1][1] * row)))
            if item[0][1] == item[1][1] and item[0][1] < cluster_nrow-1:
                for i in range(0, row):
                    cluster_edges[item].append(((item[1][0] * col - 1, item[1][1] * row + i), (item[1][0] * col, item[1][1] * row + i)))
        sub_graphs = {}
        for i,j in prd(range(0,cluster_nrow-1), range(0,cluster_ncol-1)): #need itertools product as prd
            sub_graphs[(i,j)] = RectGridGraph(row, col)
            # must be expanded such that the endges of subgraphs has the same lables as the edges of the original graph
        cluster_of_sub_graphs = cls(sub_graphs, cluster_edges)
        return cluster_of_sub_graphs
        #
         #   cluster_edges 
#int((len(graph.nodes())/(nrow * ncol))**0.5)
       

In [ ]:
#Test:
G = RectGridGraph(9, 9)
#fig = plt.figure(figsize = (25, 25))
cluster_of_sub_graphs = cluster_graph.from_rect_graph(9, 9, 3, 3)
#G.draw()
cluster_of_sub_graphs.inter_node_connections